In [1]:
import numpy as np
import time
import pandas as pd
import math
import sympy as sp
from scipy.optimize import minimize
import random
from concurrent.futures import ProcessPoolExecutor
from functools import partial

In [2]:
mus = np.load('setting/mus.npy')
mua = np.load('setting/mua.npy')
theta_0 = np.load('setting/theta_0.npy')
betaa = np.load('setting/betaa.npy')
betas = np.load('setting/betas.npy')
H = 3
num_s = 5
num_a = 1
theta_0 = 0.1*theta_0

In [3]:
#b = np.repeat(-5, H)
b = np.repeat(-534.52, H)
c = np.zeros((H, num_s))
c[H-1, 1] = 1.29

mu_s = np.zeros((H,num_s))
mu_a = np.zeros(H)
beta_s = np.zeros((H, num_s, num_s))
beta_a = np.zeros((H,num_s))
matrix_p = np.ones((H, num_a, num_s))
for t in range(3):
    mu_s[t] = mus[:,t]
    mu_a[t] = mua[0][t]
    beta_s[t] = betas[:,:,t]
    beta_a[t] = betaa[:,t]
    matrix_p[t][0] = theta_0[:,t]

In [4]:
def policy(st,mu_a_t,mu_s_t,matrix_p_t):
    at = mu_a_t+ np.dot(matrix_p_t, st-mu_s_t)
    return at

def reward(mt,bt,ct,at,st):
    rt = mt + np.dot(bt,at)+ np.dot(ct,st)
    return rt

def f(st,at,mu_s_t1,mu_s_t,mu_a_t,beta_s_t, beta_a_t):
    s_t1 = mu_s_t1 + np.dot(beta_s_t, st-mu_s_t) + np.dot(beta_a_t, at-mu_a_t)
    return s_t1

def simulate_begin(e):
    global mu_a,mu_s,beta_a,beta_s,m_fix,b,c
    for t in range(H):
        a[t] = policy(s[t], mu_a[t], mu_s[t], matrix_p[t])
        if t + 1 < H:
            s[t + 1] = f(s[t], a[t], mu_s[t + 1], mu_s[t], mu_a[t], beta_s[t], beta_a[t]) + e[(t+1)*num_s: (t+2)*num_s]
        r[t] = reward(m_fix[t], b[t], c[t], a[t], s[t])
        R[t] = R[t - 1] + r[t]
    return R[H-1]

m_fix = np.ones(H)




n_size = 20
a_final = np.zeros((n_size,H))
s_final = np.zeros((n_size,H,num_s)) # 第一个维数就是时间t

np.random.seed(0)

for i in range(n_size):
    R = np.zeros(H)
    r = np.zeros(H)
    s = np.zeros((H, num_s))
    a = np.zeros(H)
    # 确定性的参数部分
    # 这里假设s1来自于一个正态
    s[0] = np.array([0.05,0,30,5,0.7])
    #e = np.random.multivariate_normal(np.zeros(H*num_s), v_mat)
    e = np.random.normal(loc=0, scale= 0.1,size = 15)
    final_reward = simulate_begin(e)
    for j in range(H):
        a_final[i][j] = a[j]
        for k in range(num_s):
            s_final[i][j][k] = s[j][k]


np.random.seed(1)

prior_betas = np.zeros(num_s)
for i in range(num_s):
    prior_betas[i] = beta_s[0,0][i]+ np.random.normal(loc=0, scale= 0.3)

a_sample = beta_a[0] 


delta = 0.3



def sample_of_beta(subset):
    global s_final,prior_betas,period,num_s,H,beta_s,mu_s,v_mat,delta
    subset_size = len(subset)
    s_final_subset = s_final[subset]
    a_final_subset = a_final[subset]

    alpha_1 = np.zeros(subset_size)
    alpha_2 = np.zeros(subset_size)
    alpha_3 = np.zeros(subset_size)
    alpha_4 = np.zeros(subset_size)
    alpha_5 = np.zeros(subset_size)
    for i in range(subset_size):
        alpha_1[i] = s_final_subset[i][0][0] - mu_s[0][0]
        alpha_2[i] = s_final_subset[i][0][1] - mu_s[0][1]
        alpha_3[i] = s_final_subset[i][0][2] - mu_s[0][2]
        alpha_4[i] = s_final_subset[i][0][3] - mu_s[0][3]
        alpha_5[i] = s_final_subset[i][0][4] - mu_s[0][4]
    
    beta_sample = np.zeros((period,num_s))
    for i in range(num_s):
        beta_sample[0][i] = prior_betas[i]
    
    v = 0.1

    for t in range(period-1):
        m_11 = np.zeros(subset_size)
        for i in range(subset_size):
            m_11[i] = s_final_subset[i][1][0] - mu_s[1][0] - \
            beta_sample[t][1]*(s_final_subset[i][0][1] - mu_s[0][1]) - beta_sample[t][2]*(s_final_subset[i][0][2] - mu_s[0][2]) - beta_sample[t][3]*(s_final_subset[i][0][3] - mu_s[0][3]) -beta_sample[t][4]*(s_final_subset[i][0][4] - mu_s[0][4]) -\
            a_sample[0]*(a_final_subset[i][0] - mu_a[0])

        wide_beta = (delta**2*np.sum(alpha_1*m_11) + v**2*beta_s[0][0][0])/(delta**2*np.sum(alpha_1**2) + v**2)
        wide_delta2 = (delta**2*(v**2))/(delta**2*np.sum(alpha_1**2) + v**2)

        beta_sample[t+1][0] = np.random.normal(wide_beta, scale = np.sqrt(wide_delta2), size = 1)

        m_21 = np.zeros(subset_size)
        for i in range(subset_size):
            m_21[i] = s_final_subset[i][1][0] - mu_s[1][0] - \
            beta_sample[t+1][0]*(s_final_subset[i][0][0] - mu_s[0][0]) - beta_sample[t][2]*(s_final_subset[i][0][2] - mu_s[0][2])-beta_sample[t][3]*(s_final_subset[i][0][3] - mu_s[0][3]) -beta_sample[t][4]*(s_final_subset[i][0][4] - mu_s[0][4]) -\
            a_sample[0]*(a_final_subset[i][0] - mu_a[0])  

        wide_beta = (delta**2*np.sum(alpha_2*m_21) + v**2*beta_s[0][0][1])/(delta**2*np.sum(alpha_2**2) + v**2)
        wide_delta2 = (delta**2*(v**2))/(delta**2*np.sum(alpha_2**2) + v**2)
        beta_sample[t+1][1] = np.random.normal(wide_beta, scale = np.sqrt(wide_delta2), size = 1)

        m_31 = np.zeros(subset_size)
        for i in range(subset_size):
            m_31[i] = s_final[i][1][0] - mu_s[1][0] - \
            beta_sample[t+1][0]*(s_final_subset[i][0][0] - mu_s[0][0]) - beta_sample[t+1][1]*(s_final_subset[i][0][1] - mu_s[0][1])-beta_sample[t][3]*(s_final_subset[i][0][3] - mu_s[0][3]) -beta_sample[t][4]*(s_final_subset[i][0][4] - mu_s[0][4]) -\
            a_sample[0]*(a_final_subset[i][0] - mu_a[0])  

        wide_beta = (delta**2*np.sum(alpha_3*m_31) + v**2*beta_s[0][0][2])/(delta**2*np.sum(alpha_3**2) + v**2)
        wide_delta2 = (delta**2*(v**2))/(delta**2*np.sum(alpha_3**2) + v**2)
        beta_sample[t+1][2] = np.random.normal(wide_beta, scale = np.sqrt(wide_delta2), size = 1)
        
        m_41 = np.zeros(subset_size)
        for i in range(subset_size):
            m_41[i] = s_final[i][1][0] - mu_s[1][0] - \
            beta_sample[t+1][0]*(s_final_subset[i][0][0] - mu_s[0][0]) - beta_sample[t+1][1]*(s_final_subset[i][0][1] - mu_s[0][1])-beta_sample[t+1][2]*(s_final_subset[i][0][2] - mu_s[0][2]) -beta_sample[t][4]*(s_final_subset[i][0][4] - mu_s[0][4]) -\
            a_sample[0]*(a_final_subset[i][0] - mu_a[0])  

        wide_beta = (delta**2*np.sum(alpha_4*m_41) + v**2*beta_s[0][0][3])/(delta**2*np.sum(alpha_4**2) + v**2)
        wide_delta2 = (delta**2*(v**2))/(delta**2*np.sum(alpha_4**2) + v**2)
        beta_sample[t+1][3] = np.random.normal(wide_beta, scale = np.sqrt(wide_delta2), size = 1)
        
        m_51 = np.zeros(subset_size)
        for i in range(subset_size):
            m_51[i] = s_final[i][1][0] - mu_s[1][0] - \
            beta_sample[t+1][0]*(s_final_subset[i][0][0] - mu_s[0][0]) - beta_sample[t+1][1]*(s_final_subset[i][0][1] - mu_s[0][1])-beta_sample[t+1][2]*(s_final_subset[i][0][2] - mu_s[0][2]) -beta_sample[t+1][3]*(s_final_subset[i][0][3] - mu_s[0][3]) -\
            a_sample[0]*(a_final_subset[i][0] - mu_a[0])  

        wide_beta = (delta**2*np.sum(alpha_5*m_51) + v**2*beta_s[0][0][4])/(delta**2*np.sum(alpha_5**2) + v**2)
        wide_delta2 = (delta**2*(v**2))/(delta**2*np.sum(alpha_5**2) + v**2)
        beta_sample[t+1][4] = np.random.normal(wide_beta, scale = np.sqrt(wide_delta2), size = 1)
    return beta_sample[10:]

np.random.seed(2)
for i in range(3):
    s_final[i][0] += np.random.normal(0, scale = 0.1, size = num_s)
    s_final[i][1] += np.random.normal(0, scale = 0.1, size = num_s)
for i in range(3,n_size):
    s_final[i][0] += np.random.normal(0, scale = 1, size = num_s)
    s_final[i][1] += np.random.normal(0, scale = 1, size = num_s)

#s_final[14] = s_final[18]
s_final[3] = s_final[4]
s_final[17] = s_final[18]
s_final[13] = s_final[18]

In [5]:
def simulate_data(e, sample_1):
    global H,num_s,beta_s
    tmp_betas = np.zeros((H,num_s,num_s))
    for t in range(H):
        for i in range(num_s):
            for j in range(num_s):
                tmp_betas[t][i][j] = beta_s[t,i,j]
    tmp_betas[0][0] = sample_1
    R = np.zeros(H)
    r = np.zeros(H)
    s = np.zeros((H, num_s))
    a = np.zeros(H)
    s[0] = np.array([0.05,0,30,5,0.7])
    for t in range(H):
        a[t] = policy(s[t], mu_a[t], mu_s[t], matrix_p[t])
        if t + 1 < H:
            s[t + 1] = f(s[t], a[t], mu_s[t + 1], mu_s[t], mu_a[t], tmp_betas[t], beta_a[t]) + e[(t+1)*num_s: (t+2)*num_s]
        r[t] = reward(m_fix[t], b[t], c[t], a[t], s[t])
        R[t] = R[t - 1] + r[t]
    return np.round(R[H-1],4)
def value_function(subset):
    global period,H,num_s
    if len(subset) == 0:
        return 0
    else:
        sample_list = sample_of_beta(subset)
        pos_value = np.zeros(period-10)
        for i in range(period-10):
            e = np.random.normal(loc = 0 , scale = 0.1,size = 15)
            #e = np.random.multivariate_normal(np.zeros(H*num_s), v_mat)
            pos_value[i] = simulate_data(e,sample_list[i])
        res = 82.6 - np.var(pos_value, ddof =1) # 8.3
        return np.round(res,4)

def get_elements_before(lst, element):
    # 获取目标元素在列表中的索引
    if element in lst:
        index = lst.index(element)
        # 返回索引之前的所有元素
        return lst[:index]
    else:
        return []  # 如果目标元素不在列表中，返回空列表    

In [6]:
period = 900
def shapley_value(P):
    Sh_est = np.zeros(20)
    for _ in range(P):
        perm = [i for i in range(20)]
        random.shuffle(perm)
        for i in range(20):
            subset_before = get_elements_before(perm, i)
            subset_include = subset_before + [i]
            Sh_est[i] += value_function(subset_include) - value_function(subset_before)
    return Sh_est/P

s_time = time.time()
inputs = [100 for i in range(64)] # 第一个数是排列数，第二个数是cpu数
rrr = []
for _ in range(1):
    with ProcessPoolExecutor(max_workers=64) as executor:
    # 使用partial将fixed_param绑定到shapley_value函数
        results = list(executor.map(shapley_value, inputs))
    rrr.append(results)
e_time = time.time()
print('算法总耗时：', e_time-s_time)

print(len(results))
print(results[0]) #   1228.150846004486
np.savez('ps_final_5.npz', *rrr[0])
#np.savez('ps_final_3.npz', *rrr[1])
#np.savez('ps_final_3.npz', *rrr[2])

算法总耗时： 1231.703948020935
64
[ 5.66638340e+01  9.45053380e+01  1.92634918e+02  9.81140980e+01
  4.51876520e+01 -1.71800000e-02  2.04710040e+01 -7.04297290e+01
  5.53368360e+01  1.31458000e+01 -1.77155269e+02  5.66252550e+01
 -7.37810700e+00 -3.16006000e+01 -1.05067597e+02 -9.80233100e+00
  3.33814360e+01 -7.63827700e+00 -2.06262000e+00 -1.73497528e+02]
